In [1]:
from datetime import datetime
import pandas as pd
import time
from ipywidgets import widgets, Layout, Box
from IPython.display import display, clear_output

In [2]:
df_test_questions = pd.read_csv('test_quiz.csv')

In [3]:
# To try to fix the issues in the previous version, will define all the functions inside one big function

In [4]:
def create_quiz(df_questions):   
    n_questions_answered = 0
    
    def create_multipleChoice_widget(category, description, options, correct_answer, img_filepath = None):
        if correct_answer not in options:
            options.append(correct_answer)

        correct_answer_index = options.index(correct_answer)

        alphabet = 'abcdefghijklmnopqrstuvwxyz'
        options_w_letters = [f'{alphabet[i]}) {option}' for i, option in enumerate(options)]

        radio_options = [(words, i) for i, words in enumerate(options_w_letters)]

        alternativ = widgets.RadioButtons(
            options = radio_options,
            description = '',
            disabled = False,
            indent = False,
            align = 'center',
        )

        description_out = widgets.Output(layout=Layout(width='auto'))

        if img_filepath:
            image = open(img_filepath, 'rb').read()
            image_out = widgets.Image(value=image, 
                                      format='png', 
                                      width='200px', 
                                      height='auto',
                                     )

        with description_out:
            print(description)

        feedback_out = widgets.Output()

        def check_selection(b):
            a = int(alternativ.value)
            flag = a==correct_answer_index
            if flag:
                s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
            else:
                correct_answer_letter = alphabet[correct_answer_index]
                s = '\x1b[5;30;41m' + f"incorrect, the correct answer was {correct_answer_letter}" + '\x1b[0m' +"\n"
            with feedback_out:
                n_questions_answered += 1
                print(s)
                s_results = pd.Series({'Category':category, 
                                       'Question':description, 
                                       'Correct Answer':correct_answer, 
                                       'Submitted Answer':options[a], 
                                       'Score':int(flag)})
                quiz_results.append(s_results)
                alternativ.close()
                check.layout.display = 'none'
            return

        check = widgets.Button(description="check")
        check.on_click(check_selection)

        check_out = widgets.HBox([check])

        if img_filepath:
                return widgets.VBox([description_out, 
                             image_out,
                             alternativ, 
                             widgets.HBox([check]), feedback_out], 
                            layout=Layout(display='flex',
                                         flex_flow='column',
                                         align_items='stretch',
                                         width='auto')) 

        return widgets.VBox([description_out, 
                             alternativ, 
                             check_out, 
                             feedback_out], 
                            layout=Layout(
                                display='flex',
                                flex_flow='column',
                                align_items='stretch',
                                width='auto')) 
    
    def create_questions(df_questions):
        list_widgets = []
        for question_idx, s_question in df_questions.iterrows():
            category = s_question.loc['Category']
            description = s_question.loc['Question']
            options = s_question.loc['Options'].split(', ')
            correct_answer = s_question.loc['Correct Answer']
            image_filename = s_question.loc['Image Filename']

            if '.png' in str(image_filename):
                image_filepath = f'images/{image_filename}'
            else:
                image_filepath = None

            question_widget = create_multipleChoice_widget(category, description, options, correct_answer, image_filepath)
            list_widgets.append(question_widget)
        return list_widgets
    
    def on_button_clicked(b):
        if n_questions_answered == n_questions:
            df_final_results = pd.DataFrame(quiz_results)
            df_final_results['quiz'] = quiz_timestamp
            time.sleep(2)
            clear_output(wait=True)
            final_score = df_final_results['Score'].sum()
            final_score_pct = round(100 * (final_score / float(n_questions)), 2)
            final_score_message = f'Final Score: {final_score}/{n_questions} ({final_score_pct}%) \n'
            print(final_score_message)
            print(df_final_results)
    
    list_widgets = create_questions(df_questions)
    quiz_timestamp = datetime.now().strftime('%Y%m%d%M%S')
    n_questions = len(list_widgets)
    quiz_results = []    
    
    for widget in list_widgets:
        display(widget)
        for child in widget.children:
            if type(child)==widgets.widget_box.HBox:
                child.children[0].on_click(on_button_clicked)

In [5]:
create_quiz(df_test_questions)